In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from scipy import stats
import statsmodels.api as sm
import seaborn as sns
from sklearn.model_selection import train_test_split
from statsmodels.tools.eval_measures import mse
from sklearn.metrics import r2_score

In [2]:
class Data:
    def __init__(self):
        pass
    
    def consistent_tickers(self,ENV,array=[],ar=[]):
        for i in range(int(ENV.fiscalyear.min().strftime('%Y')),int(ENV.fiscalyear.max().strftime('%Y'))+1):array.append(ENV.set_index('fiscalyear')[str(i)].ticker.unique().tolist())
        for i in range(len(array)):array[i]=set(array[i])
        for i in range(len(array)):ar=set.intersection(array[i])
        return list(ar)
    
    def X(self,path='Environmental_Sector_Data.csv'):
        ENV=pd.read_csv(path,parse_dates=['Fiscal Year (fiscalyear)','Period End Date (periodenddate)'])
        #Only US
        ENV=ENV[(ENV.filter(regex='incorporation_country')=="United States").values]
        #Only Currently Operating Companies. 
        ENV=ENV[(ENV.filter(regex='status')).values==(ENV.filter(regex='status')).squeeze().unique()[:2]]
        ENV.rename(columns={'Fiscal Year (fiscalyear)':'fiscalyear','Period End Date (periodenddate)':'periodenddate','Simple Industry (simpleindustry)':'simpleindustry','Ticker (ticker)':'ticker'},inplace=True)
        cticks=self.consistent_tickers(ENV)
        ENV=ENV[ENV.ticker.isin(cticks)]
        return ENV
    
    def Y(self,path='CPIUS.csv'):
        CPI=pd.read_csv(path,parse_dates=['DATE'])
        CPI.DATE=CPI.DATE.dt.to_period('M')
        CPI.set_index('DATE',inplace=True)
        return CPI[str(int(self.X().fiscalyear.min().strftime('%Y'))):str(int(self.X().fiscalyear.max().strftime('%Y')))]
    
    def intensity_df(self):
        return self.X().filter(regex='Intensity')
    
    def absolute_df(self):
        return self.X().filter(regex='Absolute')
    
    def aggregated_df(self,flag='all',aggregation='sum'):
        if flag=='all':
            data=self.X()
        elif flag =='intensity':
            data=self.intensity_df()
        elif flag=='absolute_df':
            data=self.absolute_df()
        
        if aggregation=='sum':
            return data.sort_values('periodenddate').groupby(['fiscalyear','simpleindustry']).sum().reset_index()
        else:
            return data.sort_values('periodenddate').groupby(['fiscalyear','simpleindustry']).mean().reset_index()
    
    def transformation_df(self,outlier_removal=False,z_score_threshold=3):
        if outlier_removal:
            data=self.aggregated_df()
            data = data[(np.abs(stats.zscore(data.select_dtypes(exclude=['object','datetime64[ns]']))) < z_score_threshold).all(axis=1)]
        else:
            data=self.aggregated_df()
            
        og=data
        agg=og.select_dtypes(exclude=['object','datetime64[ns]','string'])
        single_diff=agg.diff().bfill().add_suffix('_diff')
        double_diff=agg.diff().diff().bfill().add_suffix('_doublediff')
        pct_change=agg.pct_change().bfill().add_suffix('_pct_change')
        rawdiff=(agg.diff().bfill()*agg).add_suffix('_rawxdiff')
        rawdiffdiff=(agg.diff().bfill()*agg).add_suffix('_rawxdoublediff')
        aggregation_level1=pd.concat([og,single_diff,double_diff,pct_change,rawdiff,rawdiffdiff],axis=1)
        aggregation_level2=aggregation_level1.select_dtypes(exclude=['object','datetime64[ns]','string']).apply(lambda x:x**2).add_suffix('_squared')
        aggregation_level3=aggregation_level1.select_dtypes(exclude=['object','datetime64[ns]','string']).apply(lambda x:x**3).add_suffix('_cubed')
        aggregated=pd.concat([aggregation_level1,aggregation_level2,aggregation_level3],axis=1)
        return aggregated

In [3]:
d = Data()
df = d.transformation_df()

In [4]:
finance_list=['Consumer Finance','Equity Real Estate Investment Trusts (REITs)','Insurance','Mortgage Real Estate Investment Trusts (REITs)','Trading Companies and Distributors','Diversified Financial Services','Real Estate Management and Development','Thrifts and Mortgage Finance','Capital Markets','Banks']
energy_list=['Gas Utilities','Energy Equipment and Services','Independent Power and Renewable Electricity Producers','Oil, Gas and Consumable Fuels']
ITTelcoandHW_list=['Health Care Technology','Communications Equipment','Diversified Telecommunication Services','Electrical Equipment','Electronic Equipment, Instruments and Components','Software','Technology Hardware, Storage and Peripherals','Wireless Telecommunication Services','Semiconductors and Semiconductor Equipment','Technology Hardware, Storage and Peripherals']
Infrastructureandtransporation_list=['Electric Utilities','Aerospace and Defense','Air Freight and Logistics','Airlines','Auto Components','Automobiles','Building Products','Construction Materials', 'Construction and Engineering','Road and Rail','Water Utilities']
Consumerstaples_list=['Beverages','Tobacco','Diversified Consumer Services','Household Products','Media','Paper and Forest Products','Personal Products','Professional Services','Food and Staples Retailing']
Health_list=['Health Care Equipment and Supplies','Health Care Providers and Services','Pharmaceuticals','Biotechnology']
Industrial_list=['Metals and Mining','Machinery','Chemicals','Construction Materials', 'Construction and Engineering','Industrial Conglomerates']



df_finance=df[df['simpleindustry'].isin(finance_list)].sort_values(['simpleindustry','fiscalyear'])
df_finance.simpleindustry="Finance"
df_energy=df[df['simpleindustry'].isin(energy_list)].sort_values(['simpleindustry','fiscalyear'])
df_energy.simpleindustry="Energy"
df_ITandHW=df[df['simpleindustry'].isin(ITTelcoandHW_list)].sort_values(['simpleindustry','fiscalyear'])
df_ITandHW.simpleindustry="IT, Telecom & Hardware"
df_Infrastructure=df[df['simpleindustry'].isin(Infrastructureandtransporation_list)].sort_values(['simpleindustry','fiscalyear'])
df_Infrastructure.simpleindustry="Infrastructure"
df_consumerstaples=df[df['simpleindustry'].isin(Consumerstaples_list)].sort_values(['simpleindustry','fiscalyear'])
df_consumerstaples.simpleindustry="Consumer Staples"
df_health=df[df['simpleindustry'].isin(Health_list)].sort_values(['simpleindustry','fiscalyear'])
df_health.simpleindustry="Health"
df_industrial=df[df['simpleindustry'].isin(Industrial_list)].sort_values(['simpleindustry','fiscalyear'])
df_industrial.simpleindustry="Industrial"

In [5]:
y = d.Y()
y = y.resample('A').first()
#normalized_y = np.tile(np.array((y-y.mean())/y.std()).reshape([len(y)]),len(df_finance))

In [6]:
y.to_clipboard()

In [7]:
def regression_results(df_finance,y): 
    df_finance.fiscalyear=df_finance.fiscalyear.dt.to_period('Y')
    df_finance=df_finance.merge(y,left_on='fiscalyear',right_index=True)
    df_finance.rename(columns={'CPIAUCSL':'y'},inplace=True)
    rsquared = pd.DataFrame(columns=['industry','coef','p-value','R Sq'])
    y_ = df_finance['y']
    df_finance=df_finance.drop(columns='y')
    for industry in df_finance.simpleindustry.unique():
        data = df_finance[df_finance['simpleindustry'] == industry]
        for c in data.columns[3:]:        
            if np.isinf(data[c]).any():
                continue
            model = sm.OLS(y_,sm.add_constant(data[c])).fit()
            rsquared.loc[c] = [industry,model.params[1],model.pvalues[1],model.rsquared]
        break

    rsquared = rsquared[(rsquared['p-value'] < 0.1) & (rsquared['coef'] > 0.1)].sort_values('R Sq',ascending=False)
    return rsquared

In [8]:
uv_results=pd.concat([regression_results(df_finance,y),regression_results(df_energy,y),regression_results(df_ITandHW,y),regression_results(df_Infrastructure,y),regression_results(df_consumerstaples,y),regression_results(df_health,y),regression_results(df_industrial,y)],axis=0)

In [69]:
uv_results.groupby(uv_results.index).mean().sort_values('R Sq').to_clipboard()

## Multi Regression 

In [10]:
def split(df):
    X_train, X_test = train_test_split(df, test_size=0.4, random_state=42,shuffle=False)
    return X_train.fillna(method='bfill'),X_test.fillna(method='bfill')

In [11]:
def multi_regression(df_finance,y,industry='default',size=0.1,stationarity_check=False):
    if stationarity_check:
        y=y.pct_change().fillna(method='bfill')
    
    
    
    x1 = "Absolute: Waste Nuclear (di_319551)"
    x2 = "Impact Ratio: Natural Resources Direct Cost (di_319466)_diff_squared"
    x3 = "Intensity: Waste Incineration Indirect (di_327788)_doublediff_squared"
    x4 = "Impact Ratio: Air Pollutants Indirect Cost (di_319383)_doublediff_squared"
    x5 = "Impact Ratio: GHG Direct & Indirect Cost (di_319442)_diff_squared"
    x6 = "Impact Ratio: Water Direct & Indirect Cost (di_319569)_squared"
    x7 = "Impact Ratio: Waste Direct Cost (di_319546)_diff"
    try:
        df_finance.fiscalyear=df_finance.fiscalyear.dt.to_period('Y') 
    except:
        pass
    df_finance=df_finance.merge(y,left_on='fiscalyear',right_index=True)
    df_finance.rename(columns={'CPIAUCSL':'y'},inplace=True)
    df_finance=df_finance[[x2,x3,x4,x6,x7,'y']]   
    for col in df_finance.columns:
        df_finance[col] = (df_finance[col]-df_finance[col].mean())/df_finance[col].std()
    X_train, X_test = train_test_split(df_finance, test_size=size, random_state=42,shuffle=False)
    
    
    #print(df_finance)
    
    #df_finance[[x1,x2,x3,x4,x5,x6,x7,'y','fiscalyear']]
    model1 = sm.OLS(X_train['y'],sm.add_constant(X_train[[x2,x3,x4,x6,x7]]),missing='drop').fit()
    df=model1.params.to_frame(industry).T
    df['In sample R-squared']=model1.rsquared
    #print(model1.predict(sm.add_constant(X_test[[x2,x3,x4,x6,x7]])))

    df['OOS R-squared']= r2_score(X_test['y'],model1.predict(sm.add_constant(X_test[[x2,x3,x4,x6,x7]])))
    
    
    return df,df_finance,model1



In [12]:
x=multi_regression(df,y,"All Industries")
x[0].to_clipboard()

In [14]:
finance=multi_regression(df_finance,y,'Finance')[0]
energy=multi_regression(df_energy,y,'Energy')[0]
itandhw=multi_regression(df_ITandHW,y,'IT Telecomunications and Hardware')[0]
infrastructure=multi_regression(df_Infrastructure,y,'Infrastructure')[0]
consumerstaples=multi_regression(df_consumerstaples,y,'Consumer Staples')[0]
health=multi_regression(df_health,y,'Health')[0]
industrial=multi_regression(df_industrial,y,'Industrial')[0]

In [15]:
results=pd.concat([finance,energy,itandhw,infrastructure,consumerstaples,health,industrial],axis=0)
results.to_clipboard()

In [61]:
def backtesting(df_finance):
        f=finance.iloc[:,:6]
        findf=multi_regression(df_finance,y,'Finance')[1]
        findf['const']=1
        findf.drop(columns='y').reindex(columns=f.columns) 
        dump=f.to_numpy()*findf.drop(columns='y').reindex(columns=f.columns)
        return dump.sum(axis=1)
        

In [68]:
df_test=pd.concat([backtesting(df_finance),backtesting(df_energy),backtesting(df_ITandHW),backtesting(df_Infrastructure),backtesting(df_consumerstaples),backtesting(df_health),backtesting(df_industrial)],axis=0)
df_test.to_clipboard()